In [1]:
# !pip install --upgrade tensorflow
!pip install tensorflow==2.15
!pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      

In [2]:
!pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


In [3]:
# !pip install --upgrade transformers

In [4]:
# !pip show tensorflow
# !pip show Keras

In [5]:
import os
import tensorflow as tf
from transformers import TFElectraForSequenceClassification, ElectraTokenizer
from datasets import load_dataset, load_metric
from google.colab import drive

In [6]:
drive.mount('/content/drive')

# Checkpoints and final model
checkpoint_dir = '/content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-qqp'
final_model_path = '/content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-qqp-final'

Mounted at /content/drive


In [7]:
# https://colab.research.google.com/drive/1VoWyzlXZVR5sNygejktJ4Uguw3HIZckb?usp=sharing

# Load ELECTRA BASE model and tokenizer
model_name = "google/electra-base-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = TFElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load and preprocess the GLUE QQP dataset
# https://huggingface.co/docs/datasets/v1.1.2/loading_datasets.html
# https://huggingface.co/docs/datasets/en/process
# https://huggingface.co/datasets/nyu-mll/glue
dataset = load_dataset("glue", "qqp")
metric = load_metric("glue", "qqp")

# encoded_dataset = dataset.map(lambda examples: tokenizer(examples["question1"],
#                                                          examples["question2"],
#                                                          truncation=True,
#                                                          padding="max_length",
#                                                          return_tensors="tf"),
#                               batched=True)
def tokenize_and_encode(examples):
    tokenized_examples = tokenizer(examples["question1"],
                                   examples["question2"],
                                   truncation=True,
                                   padding="max_length",
                                   return_tensors="tf")
    tokenized_examples["label"] = tf.one_hot(examples["label"], depth=2)
    return tokenized_examples

encoded_dataset = dataset.map(tokenize_and_encode, batched=True)


# Convert to TensorFlow dataset
# https://huggingface.co/docs/datasets/v1.16.1/use_dataset.html
train_data = encoded_dataset["train"].to_tf_dataset(columns=["attention_mask", "input_ids", "token_type_ids"],
                                                    label_cols=["label"],
                                                    shuffle=True,
                                                    batch_size=32)

validation_data = encoded_dataset["validation"].to_tf_dataset(columns=["attention_mask", "input_ids", "token_type_ids"],
                                                              label_cols=["label"],
                                                              shuffle=False,
                                                              batch_size=32)

# Checkpoints
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'checkpoint_epoch_{epoch}.weights.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Hyperparameters from the ELECTRA paper https://arxiv.org/abs/2003.10555
# "Table 6: Fine-tune hyperparameters"
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,
                                     epsilon=1e-6,
                                     beta_1=0.9,
                                     beta_2=0.999)

# Compile
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Finetune
model.fit(train_data, validation_data=validation_data, epochs=3, callbacks=[checkpoint_callback])

# Save the final model
model.save_pretrained(final_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

...

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
11371/11371 [==============================] - ETA: 0s - loss: 0.2819 - accuracy: 0.8780
Epoch 1: val_loss improved from inf to 0.24496, saving model to /content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-qqp/checkpoint_epoch_1.weights.h5


/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


11371/11371 [==============================] - 5067s 439ms/step - loss: 0.2819 - accuracy: 0.8780 - val_loss: 0.2450 - val_accuracy: 0.8945
Epoch 2/3
11371/11371 [==============================] - ETA: 0s - loss: 0.2040 - accuracy: 0.9173
Epoch 2: val_loss improved from 0.24496 to 0.23827, saving model to /content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-qqp/checkpoint_epoch_2.weights.h5
11371/11371 [==============================] - 4982s 438ms/step - loss: 0.2040 - accuracy: 0.9173 - val_loss: 0.2383 - val_accuracy: 0.9094
Epoch 3/3
11371/11371 [==============================] - ETA: 0s - loss: 0.1586 - accuracy: 0.9382
Epoch 3: val_loss improved from 0.23827 to 0.22670, saving model to /content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-qqp/checkpoint_epoch_3.weights.h5
11371/11371 [==============================] - 4981s 438ms/step - loss: 0.1586 - accuracy: 0.9382 - val_loss: 0.2267 - val_accuracy: 0.9096


- ELECTRA BASE fine tuned on GLUE Quora Question Pairs to determine whether a pair of questions are semantically equivalent.
- Ran in Google Colab Pro+ on A100 GPU
  - System RAM 5.7B
  - GPU RAM 32.5GB
  - Ran in about 251 minutes
  - Final model weights saved
  - loss: 0.1586 - accuracy: 0.9382 - val_loss: 0.2267 - val_accuracy: 0.9096
- Did not implement the the warmup and decay, and instead used a common learning rate of 5e-5
- Next step: Benchmark against GLUE QQP